In [ ]:
!pip install pandas openpyxl gspread oauth2client


   ---------------------------------------- 0.0/59.9 kB ? eta -:--:--
   -------------------- ------------------- 30.7/59.9 kB 1.3 MB/s eta 0:00:01
   -------------------- ------------------- 30.7/59.9 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 59.9/59.9 kB 455.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ------------------------------------- -- 92.2/98.2 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 98.2/98.2 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/212.3 kB ? eta -:--:--
   ----------------------- ---------------- 122.9/212.3 kB 3.6 MB/s eta 0:00:01
   ----------------------- ---------------- 122.9/212.3 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 212.3/212.3 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
   ---------------------------------------- 96.9/96.9 kB 2.8 MB/s eta 0:00:00
   ------

In [5]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 1. Прочитаем основной файл (statement) и файл фильтра (filterl)
data_df = pd.read_excel('statement.xlsx')
filter_df = pd.read_excel('filterl.xlsx')

# 2. Получаем список значений для фильтрации из 4-го столбца файла filterl
filter_values = filter_df.iloc[:, 0].tolist()

# 3. Фильтруем данные: исключаем строки, где 4-й столбец есть в списке фильтра
filtered_df = data_df[~data_df.iloc[:, 3].isin(filter_values)]

# 4. Оставляем только строки, где во 2-м столбце значение больше 0
filtered_df.iloc[:, 1] = pd.to_numeric(filtered_df.iloc[:, 1], errors='coerce')
filtered_df = filtered_df[filtered_df.iloc[:, 1] > 0]

# 5. Преобразуем дату в формат ДД.ММ.ГГГГ
filtered_df.iloc[:, 0] = pd.to_datetime(filtered_df.iloc[:, 0], errors='coerce').dt.strftime('%d.%m.%Y')

# 6. Остальные колонки (кроме суммы) — в строки
for col in range(2, filtered_df.shape[1]):
    filtered_df.iloc[:, col] = filtered_df.iloc[:, col].astype(str)

# *** ДОБАВИМ ПРОВЕРКУ ***
print("Размер filtered_df:", filtered_df.shape)
print("Первые строки filtered_df:")
print(filtered_df.head())

# 7. Авторизация для доступа к Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
client = gspread.authorize(credentials)

# 8. Открываем нужный лист в Google Sheets
spreadsheet = client.open('Priut')
worksheet = spreadsheet.worksheet('Income')

# 9. Очищаем диапазон A5:F (если нужно)
worksheet.batch_clear([f"A5:F{worksheet.row_count}"])

# 10. Загружаем отфильтрованные данные с заголовками (начиная с 5-й строки)
worksheet.update(filtered_df.values.tolist(), 'A5')


print("Данные успешно обновлены!")


Размер filtered_df: (481, 4)
Первые строки filtered_df:
    Unnamed: 0 Unnamed: 1            Unnamed: 2            Unnamed: 3
7   01.01.2025     1000.0            Пополнение             Мадина И.
22  04.01.2025      500.0            Пополнение             Мадина И.
41  07.01.2025     5000.0  Поступление перевода  Поступление перевода
54  09.01.2025     5000.0            Пополнение            Меруерт К.
57  09.01.2025     2000.0            Пополнение             Мадина И.
Данные успешно обновлены!
